In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
total = pd.read_csv("../data/totalContribs_all_years_new_hashtags_or_user_ids.csv")
display(total)

,year,corporation,edits,contributors
0,2024,WIGeoGIS-OMV,245,1
1,2017,Wonder,5147,1
2,2018,Wonder,17006,1
3,2019,Wonder,13031,1
4,2020,Wonder,36808,1
...,...,...,...,...
299,2019,Neshan,197477,8
300,2020,Neshan,171865,8
301,2021,Neshan,72564,11
302,2022,Neshan,199791,12


## Corporate Edits and Contributors

In [8]:
corporate_tot = total.loc[total['corporation'] != 'nc'].groupby(['year']).sum()
nc_tot = total.loc[total['corporation'] == 'nc']
corporate_tot['corporate'] = 'CE'
corporate_tot = corporate_tot.drop(columns='corporation')
corporate_tot = corporate_tot.rename(columns={'edits':'edits_CE', 'contributors': 'contributors_CE' })

display(corporate_tot)

,edits_CE,contributors_CE,corporate
year,,,
2008,18442,1,CE
2009,6180,3,CE
2010,71193,3,CE
2011,144523,8,CE
2012,543698,11,CE
2013,1037609,16,CE
2014,736059,32,CE
2015,1454201,208,CE
2016,2611413,532,CE


## Combined: Corporate and Non-Corporate Edits and Contributors

In [9]:
nc_tot = nc_tot.rename(columns={'edits':'edits_NCE', 'contributors': 'contributors_NCE' }).drop(columns='corporation')
combi = nc_tot.merge(corporate_tot, on = 'year', how = 'left')
combi = combi.fillna(0)
combi['edits_total'] = combi['edits_NCE'] + combi['edits_CE']
combi['contributors_total'] = combi['contributors_NCE'] + combi['contributors_CE']

combi['contributors_percentage'] = (combi['contributors_CE'] / combi['contributors_total'])*100
combi['contributors_edits'] = (combi['edits_CE'] / combi['edits_total'])*100
#combi.to_csv('../data/fig1_2005-2024_yearly_edits_contributors_new_data.csv')

display(combi)

,year,edits_NCE,contributors_NCE,edits_CE,contributors_CE,corporate,edits_total,contributors_total,contributors_percentage,contributors_edits
0,2005,30653,50,0.0,0.0,0,30653.0,50.0,0.000000,0.000000
1,2006,3817776,410,0.0,0.0,0,3817776.0,410.0,0.000000,0.000000
2,2007,188409659,5070,0.0,0.0,0,188409659.0,5070.0,0.000000,0.000000
3,2008,114289998,29177,18442.0,1.0,CE,114308440.0,29178.0,0.003427,0.016134
4,2009,302885106,59417,6180.0,3.0,CE,302891286.0,59420.0,0.005049,0.002040
5,2010,127327684,70763,71193.0,3.0,CE,127398877.0,70766.0,0.004239,0.055882
6,2011,132498526,90505,144523.0,8.0,CE,132643049.0,90513.0,0.008839,0.108956
7,2012,151467686,119306,543698.0,11.0,CE,152011384.0,119317.0,0.009219,0.357669
8,2013,142238861,125098,1037609.0,16.0,CE,143276470.0,125114.0,0.012788,0.724201
9,2014,180689409,153500,736059.0,32.0,CE,181425468.0,153532.0,0.020843,0.405709


## Stats per Company

In [10]:
def prepYear(t1, year, stats_type):
    df = t1.get_group(year)
    df = df[['corporation', stats_type]].sort_values(stats_type, ascending = False).set_index('corporation').rename(columns={stats_type: year})
    df = df.reset_index()
    return df

In [11]:
test = total.copy()
t1 = test.groupby('year')
y20 = t1.get_group(2020)

years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024] 
frames = []

for year in years:
    df = prepYear(t1, year, 'edits')
    frames.append(df)

In [12]:
corporation_combi = pd.merge(frames[0], frames[1], on = 'corporation', how = 'outer')
corporation_combi = pd.merge(corporation_combi, frames[2], on = 'corporation', how = 'outer')
corporation_combi = pd.merge(corporation_combi, frames[3], on = 'corporation', how = 'outer')
corporation_combi = pd.merge(corporation_combi, frames[4], on = 'corporation', how = 'outer')
corporation_combi = pd.merge(corporation_combi, frames[5], on = 'corporation', how = 'outer')
corporation_combi = pd.merge(corporation_combi, frames[6], on = 'corporation', how = 'outer')
corporation_combi = pd.merge(corporation_combi, frames[7], on = 'corporation', how = 'outer')
corporation_combi = pd.merge(corporation_combi, frames[8], on = 'corporation', how = 'outer')

corporation_combi_tr = corporation_combi.set_index('corporation').transpose().reset_index().rename(columns={'index': 'year'})
#corporation_combi.to_csv('../data/fig1_absolute_edits_per_corporation_16-23_new_dataset.csv', header = True)

### Overall Edits per Company

In [13]:
timeline = corporation_combi.copy()
timeline = timeline.rename_axis('year', axis=1)
timeline = timeline.set_index('corporation')
timeline = timeline.fillna(0)
timeline = timeline.apply(pd.to_numeric, errors ='coerce')

t1 = timeline.transpose()
t1.drop(columns={'nc'}).sum().sort_values(ascending = False)

corporation
Apple              39126338.0
Kaart              39090390.0
Meta               36192382.0
Amazon             19794827.0
DigitalEgypt        8688469.0
Grab                6536874.0
Microsoft           4200871.0
Lyft                2891920.0
TomTom              2802104.0
Telenav             1728748.0
DevelopmentSeed     1580741.0
Mapbox              1527727.0
Neshan               987078.0
Uber                 823227.0
RocketData           507861.0
Bolt                 469240.0
Graphmasters         402844.0
Lightcyphers         396910.0
Snapp                227011.0
Wonder               149731.0
TfNWS                135488.0
MeNextBillion        109015.0
TIDBO                 96260.0
Expedia               83959.0
Snap                  64692.0
Stackbox              50047.0
Kontur                46199.0
Gojek                  7592.0
AppLogica              4977.0
WIGeoGIS-OMV           2628.0
Komoot                 1390.0
Balad                  1001.0
dtype: float64

### Yearly Edits per Company

In [14]:
all_company_names = list(t1.columns)
all_company_names.remove('nc')

large_company_names = [
    "Amazon",
    "Apple",
    "DigitalEgypt",
    "Grab",
    "Kaart",
    "Lyft",
    "Mapbox",
    "Meta",
    "Microsoft",
    "TomTom"
]

other_company_names = list(set(all_company_names) - set(large_company_names))
print(large_company_names)
print(other_company_names)

['Amazon', 'Apple', 'DigitalEgypt', 'Grab', 'Kaart', 'Lyft', 'Mapbox', 'Meta', 'Microsoft', 'TomTom']
['Komoot', 'TIDBO', 'Snap', 'Kontur', 'Telenav', 'RocketData', 'Neshan', 'AppLogica', 'MeNextBillion', 'Lightcyphers', 'DevelopmentSeed', 'TfNWS', 'Balad', 'Graphmasters', 'Snapp', 'Gojek', 'WIGeoGIS-OMV', 'Stackbox', 'Wonder', 'Bolt', 'Expedia', 'Uber']


In [15]:
Other = timeline.copy()
Other = Other.transpose()

Other = Other[other_company_names]
Other.transpose().sum()

year
2016     754740.0
2017     704296.0
2018    1732799.0
2019    1014456.0
2020     951468.0
2021     573980.0
2022    1039728.0
2023     766431.0
2024     338741.0
dtype: float64

In [16]:
large = timeline.transpose().reset_index()
large = large[['year', 'Meta', 'Apple', 'Kaart', 'Amazon', 'TomTom', 'Microsoft', 'DigitalEgypt', 'Grab', 'Mapbox', 'Lyft']]

Other = timeline.copy()
Other = Other.transpose()
Other = Other[other_company_names]
large['Other'] = Other.transpose().sum().values

display(large)

corporation,year,Meta,Apple,Kaart,Amazon,TomTom,Microsoft,DigitalEgypt,Grab,Mapbox,Lyft,Other
0,2016,5067.0,5834.0,1701595.0,38324.0,552.0,69.0,0.0,30854.0,74378.0,0.0,754740.0
1,2017,447901.0,125040.0,2216957.0,240667.0,103501.0,10362.0,0.0,241506.0,65323.0,0.0,704296.0
2,2018,2475779.0,1076907.0,3734157.0,541214.0,11130.0,311349.0,1630.0,476204.0,27702.0,18211.0,1732799.0
3,2019,4360806.0,2065643.0,8568149.0,3985206.0,2.0,766491.0,2919993.0,736532.0,99230.0,342657.0,1014456.0
4,2020,6930498.0,14408303.0,8084299.0,7223019.0,25.0,1113810.0,2279826.0,899725.0,223755.0,125015.0,951468.0
5,2021,10685781.0,12152428.0,5503095.0,6073283.0,561052.0,833763.0,925386.0,2193255.0,417451.0,206435.0,573980.0
6,2022,7984176.0,3776583.0,4633095.0,939266.0,710657.0,444824.0,1202121.0,1056829.0,313319.0,805473.0,1039728.0
7,2023,3118345.0,3414275.0,3059998.0,612303.0,1015671.0,469489.0,833052.0,652006.0,229354.0,932985.0,766431.0
8,2024,184029.0,2101325.0,1589045.0,141545.0,399514.0,250714.0,526461.0,249963.0,77215.0,461144.0,338741.0


### corporate percentage

In [17]:
# calculating the total number of PFs per year - taking the df created for the first graph
newTotal = combi.copy()
total_edits = newTotal[['year', 'edits_total']]
total_edits = total_edits.tail(9).head(9)
# adding the total corporate numbers
top10List = pd.merge(total_edits, large, on = 'year', how ='left')

In [18]:
top10List

,year,edits_total,Meta,Apple,Kaart,Amazon,TomTom,Microsoft,DigitalEgypt,Grab,Mapbox,Lyft,Other
0,2016,164003085.0,5067.0,5834.0,1701595.0,38324.0,552.0,69.0,0.0,30854.0,74378.0,0.0,754740.0
1,2017,192734145.0,447901.0,125040.0,2216957.0,240667.0,103501.0,10362.0,0.0,241506.0,65323.0,0.0,704296.0
2,2018,224234225.0,2475779.0,1076907.0,3734157.0,541214.0,11130.0,311349.0,1630.0,476204.0,27702.0,18211.0,1732799.0
3,2019,228368312.0,4360806.0,2065643.0,8568149.0,3985206.0,2.0,766491.0,2919993.0,736532.0,99230.0,342657.0,1014456.0
4,2020,280866625.0,6930498.0,14408303.0,8084299.0,7223019.0,25.0,1113810.0,2279826.0,899725.0,223755.0,125015.0,951468.0
5,2021,300640586.0,10685781.0,12152428.0,5503095.0,6073283.0,561052.0,833763.0,925386.0,2193255.0,417451.0,206435.0,573980.0
6,2022,267950955.0,7984176.0,3776583.0,4633095.0,939266.0,710657.0,444824.0,1202121.0,1056829.0,313319.0,805473.0,1039728.0
7,2023,290348464.0,3118345.0,3414275.0,3059998.0,612303.0,1015671.0,469489.0,833052.0,652006.0,229354.0,932985.0,766431.0
8,2024,183562172.0,184029.0,2101325.0,1589045.0,141545.0,399514.0,250714.0,526461.0,249963.0,77215.0,461144.0,338741.0


In [19]:
corpor10 = ['Meta', 'Apple', 'Kaart', 'Amazon', 'TomTom', 'Microsoft', 'DigitalEgypt', 'Grab', 'Mapbox', 'Lyft', 'Other']
newDF = top10List.copy()

In [20]:
def calc_perc_edits(df, column):
    df[column] = (df[column] / df['edits_total']) * 100
    return df

In [21]:
for tags in corpor10: 
    calc_perc_edits(newDF, tags)

newDF = newDF.drop(columns={'edits_total'})
#newDF.to_csv('../data/fig1_top10_corpos_percentage_edits_based_on_total_edits_newData.csv', header = True)

display(newDF)

,year,Meta,Apple,Kaart,Amazon,TomTom,Microsoft,DigitalEgypt,Grab,Mapbox,Lyft,Other
0,2016,0.003090,0.003557,1.037538,0.023368,3.365790e-04,0.000042,0.000000,0.018813,0.045352,0.000000,0.460199
1,2017,0.232393,0.064877,1.150267,0.124870,5.370143e-02,0.005376,0.000000,0.125305,0.033893,0.000000,0.365424
2,2018,1.104104,0.480260,1.665293,0.241361,4.963560e-03,0.138850,0.000727,0.212369,0.012354,0.008121,0.772763
3,2019,1.909550,0.904523,3.751899,1.745078,8.757782e-07,0.335638,1.278633,0.322519,0.043452,0.150046,0.444219
4,2020,2.467541,5.129945,2.878341,2.571690,8.901022e-06,0.396562,0.811711,0.320339,0.079666,0.044510,0.338762
5,2021,3.554337,4.042178,1.830456,2.020114,1.866188e-01,0.277329,0.307805,0.729527,0.138854,0.068665,0.190919
6,2022,2.979715,1.409431,1.729083,0.350537,2.652191e-01,0.166009,0.448635,0.394411,0.116931,0.300605,0.388029
7,2023,1.074001,1.175923,1.053905,0.210886,3.498110e-01,0.161698,0.286915,0.224560,0.078993,0.321333,0.263969
8,2024,0.100254,1.144748,0.865671,0.077110,2.176451e-01,0.136583,0.286803,0.136173,0.042065,0.251220,0.184537


## Contributors

### Contributors Combined Table

In [22]:
test = total.copy()
t1 = test.groupby('year')
y20 = t1.get_group(2020)

dat = {'corporation' : []}
contributors_combi = pd.DataFrame(data=dat)

for year in years:
    df = prepYear(t1, year, 'contributors')
    contributors_combi = pd.merge(contributors_combi, df, on = 'corporation', how = 'outer')

display(contributors_combi)
#contributors_combi.to_csv('../data/fig1_absolute_contributors_per_corporation_16-23_new_dataset.csv', header = True)

,corporation,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,nc,259711.0,315941.0,305230.0,279117.0,299809.0,288153.0,255001,263916.0,198745.0
1,Amazon,291.0,912.0,432.0,722.0,1081.0,2323.0,1479,1197.0,570.0
2,Mapbox,83.0,64.0,31.0,34.0,80.0,95.0,104,109.0,78.0
3,Expedia,69.0,186.0,106.0,3.0,1.0,8.0,12,13.0,9.0
4,Kaart,46.0,72.0,332.0,456.0,417.0,365.0,167,118.0,157.0
5,Telenav,12.0,21.0,27.0,30.0,9.0,18.0,16,15.0,1.0
6,TomTom,9.0,641.0,2.0,1.0,2.0,106.0,280,124.0,96.0
7,DevelopmentSeed,4.0,4.0,4.0,5.0,4.0,3.0,5,5.0,5.0
8,Grab,3.0,96.0,177.0,464.0,182.0,427.0,314,250.0,129.0
9,Apple,3.0,33.0,67.0,103.0,1121.0,1151.0,573,460.0,371.0


In [27]:
contributors_combi.set_index('corporation').transpose().sum().sort_values(ascending = False)

corporation
nc                 2465623.0
Amazon                9007.0
Apple                 3882.0
Kaart                 2130.0
Grab                  2042.0
TomTom                1261.0
Meta                  1002.0
Mapbox                 678.0
Expedia                407.0
Microsoft              208.0
Uber                   205.0
Lyft                   188.0
Bolt                   156.0
Telenav                149.0
DigitalEgypt           138.0
Neshan                  67.0
Snapp                   53.0
Graphmasters            40.0
DevelopmentSeed         39.0
Lightcyphers            32.0
MeNextBillion           30.0
Stackbox                29.0
Snap                    26.0
TfNWS                   26.0
Gojek                   25.0
RocketData              14.0
TIDBO                   12.0
Komoot                  12.0
Kontur                  10.0
WIGeoGIS-OMV             9.0
Wonder                   9.0
Balad                    8.0
AppLogica                2.0
dtype: float64

In [31]:
large_company_names_contribs = [
    "Amazon",
    "Apple",
    "Kaart",
    "Grab",
    "TomTom",
    "Meta",
    "Mapbox",
    "Expedia",
    "Microsoft",
    "Uber"
]

other_company_names_contribs = list(set(all_company_names) - set(large_company_names_contribs))
print(large_company_names_contribs)
print(other_company_names_contribs)

['Amazon', 'Apple', 'Kaart', 'Grab', 'TomTom', 'Meta', 'Mapbox', 'Expedia', 'Microsoft', 'Uber']
['Komoot', 'TIDBO', 'Snap', 'DigitalEgypt', 'Kontur', 'Telenav', 'RocketData', 'Neshan', 'AppLogica', 'MeNextBillion', 'Lightcyphers', 'DevelopmentSeed', 'TfNWS', 'Balad', 'Graphmasters', 'Lyft', 'Snapp', 'Gojek', 'WIGeoGIS-OMV', 'Stackbox', 'Wonder', 'Bolt']


### Contribuors final table prep

In [28]:
contribs_timeline = contributors_combi.copy()
contribs_timeline = contribs_timeline.rename_axis('year', axis=1)
contribs_timeline = contribs_timeline.set_index('corporation')

contribs_timeline = contribs_timeline.fillna(0)
contribs_timeline = contribs_timeline.apply(pd.to_numeric, errors ='coerce')
c1 = contribs_timeline.transpose()

display(c1)

corporation,nc,Amazon,Mapbox,Expedia,Kaart,Telenav,TomTom,DevelopmentSeed,Grab,Apple,...,TIDBO,Balad,Kontur,DigitalEgypt,MeNextBillion,Gojek,RocketData,Stackbox,Komoot,AppLogica
year,,,,,,,,,,,,,,,,,,,,,
2016,259711.0,291.0,83.0,69.0,46.0,12.0,9.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017,315941.0,912.0,64.0,186.0,72.0,21.0,641.0,4.0,96.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018,305230.0,432.0,31.0,106.0,332.0,27.0,2.0,4.0,177.0,67.0,...,3.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,279117.0,722.0,34.0,3.0,456.0,30.0,1.0,5.0,464.0,103.0,...,5.0,2.0,1.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0
2020,299809.0,1081.0,80.0,1.0,417.0,9.0,2.0,4.0,182.0,1121.0,...,1.0,0.0,2.0,20.0,28.0,2.0,1.0,1.0,1.0,0.0
2021,288153.0,2323.0,95.0,8.0,365.0,18.0,106.0,3.0,427.0,1151.0,...,0.0,2.0,2.0,24.0,1.0,5.0,4.0,0.0,2.0,0.0
2022,255001.0,1479.0,104.0,12.0,167.0,16.0,280.0,5.0,314.0,573.0,...,1.0,2.0,2.0,27.0,1.0,12.0,5.0,12.0,2.0,2.0
2023,263916.0,1197.0,109.0,13.0,118.0,15.0,124.0,5.0,250.0,460.0,...,1.0,1.0,2.0,24.0,0.0,3.0,2.0,12.0,3.0,0.0
2024,198745.0,570.0,78.0,9.0,157.0,1.0,96.0,5.0,129.0,371.0,...,1.0,0.0,0.0,20.0,0.0,3.0,2.0,4.0,4.0,0.0


In [ ]:
Amazon                9007.0
Apple                 3882.0
Kaart                 2130.0
Grab                  2042.0
TomTom                1261.0
Meta                  1002.0
Mapbox                 678.0
Expedia                407.0
Microsoft              208.0
Uber                   205.0

In [32]:
large_contribs = contribs_timeline.transpose().reset_index()
large_contribs = large_contribs[['year', 'Amazon', 'Apple', 'Kaart', 'Grab', 'TomTom', 'Meta', 'Mapbox', 'Expedia', 'Microsoft', 'Uber']]

Other = contribs_timeline.copy()
Other = Other.transpose()
Other = Other[other_company_names_contribs]
large_contribs['Other'] = Other.transpose().sum().values

display(large_contribs)

corporation,year,Amazon,Apple,Kaart,Grab,TomTom,Meta,Mapbox,Expedia,Microsoft,Uber,Other
0,2016,291.0,3.0,46.0,3.0,9.0,1.0,83.0,69.0,1.0,0.0,26.0
1,2017,912.0,33.0,72.0,96.0,641.0,24.0,64.0,186.0,3.0,2.0,40.0
2,2018,432.0,67.0,332.0,177.0,2.0,85.0,31.0,106.0,12.0,87.0,86.0
3,2019,722.0,103.0,456.0,464.0,1.0,123.0,34.0,3.0,14.0,77.0,145.0
4,2020,1081.0,1121.0,417.0,182.0,2.0,177.0,80.0,1.0,47.0,21.0,155.0
5,2021,2323.0,1151.0,365.0,427.0,106.0,169.0,95.0,8.0,41.0,7.0,168.0
6,2022,1479.0,573.0,167.0,314.0,280.0,218.0,104.0,12.0,34.0,7.0,187.0
7,2023,1197.0,460.0,118.0,250.0,124.0,148.0,109.0,13.0,32.0,3.0,163.0
8,2024,570.0,371.0,157.0,129.0,96.0,57.0,78.0,9.0,24.0,1.0,104.0


### Contributors percentage table

In [33]:
# calculating the total number of PFs per year - taking the df created for the first graph
newTotal = combi.copy()
total_contribs = newTotal[['year', 'contributors_total']]
total_contribs = total_contribs.tail(9).head(9)

# adding the total corporate numbers
top10List_contribs = pd.merge(total_contribs, large_contribs, on = 'year', how ='left')

display(top10List_contribs)

,year,contributors_total,Amazon,Apple,Kaart,Grab,TomTom,Meta,Mapbox,Expedia,Microsoft,Uber,Other
0,2016,260243.0,291.0,3.0,46.0,3.0,9.0,1.0,83.0,69.0,1.0,0.0,26.0
1,2017,318014.0,912.0,33.0,72.0,96.0,641.0,24.0,64.0,186.0,3.0,2.0,40.0
2,2018,306647.0,432.0,67.0,332.0,177.0,2.0,85.0,31.0,106.0,12.0,87.0,86.0
3,2019,281259.0,722.0,103.0,456.0,464.0,1.0,123.0,34.0,3.0,14.0,77.0,145.0
4,2020,303093.0,1081.0,1121.0,417.0,182.0,2.0,177.0,80.0,1.0,47.0,21.0,155.0
5,2021,293013.0,2323.0,1151.0,365.0,427.0,106.0,169.0,95.0,8.0,41.0,7.0,168.0
6,2022,258376.0,1479.0,573.0,167.0,314.0,280.0,218.0,104.0,12.0,34.0,7.0,187.0
7,2023,266533.0,1197.0,460.0,118.0,250.0,124.0,148.0,109.0,13.0,32.0,3.0,163.0
8,2024,200341.0,570.0,371.0,157.0,129.0,96.0,57.0,78.0,9.0,24.0,1.0,104.0


In [35]:
corpor10_contribs = ['Amazon', 'Apple', 'Kaart', 'Grab', 'TomTom', 'Meta', 'Mapbox', 'Expedia', 'Microsoft', 'Uber', 'Other']
contirb_perc = top10List_contribs.copy()

def calc_perc(df, column):
    df[column] = (df[column] / df['contributors_total']) * 100
    return df

for tags in corpor10_contribs: 
    calc_perc(contirb_perc, tags)

contirb_perc = contirb_perc.drop(columns = {'contributors_total'})
contirb_perc.to_csv('../data/fig1_top10_corpos_percentage_contributors_based_on_total_edits_newData_updateOrder.csv', header = True)

display(contirb_perc)

,year,Amazon,Apple,Kaart,Grab,TomTom,Meta,Mapbox,Expedia,Microsoft,Uber,Other
0,2016,0.111819,0.001153,0.017676,0.001153,0.003458,0.000384,0.031893,0.026514,0.000384,0.000000,0.009991
1,2017,0.286780,0.010377,0.022641,0.030187,0.201563,0.007547,0.020125,0.058488,0.000943,0.000629,0.012578
2,2018,0.140879,0.021849,0.108268,0.057721,0.000652,0.027719,0.010109,0.034567,0.003913,0.028371,0.028045
3,2019,0.256703,0.036621,0.162128,0.164972,0.000356,0.043732,0.012089,0.001067,0.004978,0.027377,0.051554
4,2020,0.356656,0.369853,0.137582,0.060048,0.000660,0.058398,0.026395,0.000330,0.015507,0.006929,0.051139
5,2021,0.792798,0.392815,0.124568,0.145727,0.036176,0.057677,0.032422,0.002730,0.013993,0.002389,0.057335
6,2022,0.572422,0.221770,0.064634,0.121528,0.108369,0.084373,0.040251,0.004644,0.013159,0.002709,0.072375
7,2023,0.449100,0.172587,0.044272,0.093797,0.046523,0.055528,0.040895,0.004877,0.012006,0.001126,0.061156
8,2024,0.284515,0.185184,0.078366,0.064390,0.047918,0.028451,0.038934,0.004492,0.011980,0.000499,0.051911
